In [22]:
#image analysis
import skimage.io
import imageio
import alienlab.plot
from alienlab.improcessing import normalize, grey_to_rgb, make_binary
import alienlab.segment
from alienlab.fo import FramesOperator
import alienlab.io
from scipy import optimize
import glob
from alienlab.regression_func import *
import copy
from VoltageIntensityClass import VoltageIntensity
from tqdm import tqdm

from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects
from skimage.transform import resize

import tifffile as tiff

#interactive widget packages
from ipywidgets import interact, interactive, fixed, interact_manual
from tkinter.filedialog import askopenfilename, askdirectory

from VoltageIntensityClass import VoltageIntensity

import time
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

%matplotlib ipympl
import ipywidgets as wdg  # Using the ipython notebook widgets

#%matplotlib inline


def residuals(parameters,x_data,y_observed,func):
    '''
    Compute residuals of y_predicted - y_observed
    where:
    y_predicted = func(parameters,x_data)
    '''
    return func(parameters,x_data) - y_observed

def clip(input_image, high = 95, low = 5):
    im = copy.copy(input_image)
    m = np.median(im)
    im[im<np.percentile(im, low)]=np.percentile(im, low)
    im[im>np.percentile(im, high)]=np.percentile(im, high)
    return im

def platt(parameters, xdata):
    M = parameters[0]
    alpha = parameters[1]
    return M*(1- np.exp(-alpha*xdata/M))

def exp_decay(parameters, xdata):
    '''
    Calculate an exponetial decay of the form:
    S= a * exp(-xdata/b)
    '''
    A = parameters[0]
    tau = parameters[1]
    y0 = parameters[2]
    return A * np.exp(-xdata/tau) + y0

def modele_direct(parameters, x_data):
    a = parameters[0]
    b = parameters[1]
    c = parameters[2]
    d = parameters[3]
    e = parameters[4]
    return (a*x_data +b*np.sqrt(c*I**2+d*I+1)+e)/I

def modele_inverse(parameters, x_data):
    a = parameters[0]
    b = parameters[1]
    c = parameters[2]
    d = parameters[3]
    e = parameters[4]
    f = parameters[5]
    return a*I + b*np.sqrt(c*I**2+d*I+1) + e

In [23]:
#file_path = "ENTER NAME OF THE FILE YOU DROPPED IN THE FILE SECTION HERE"
file_folder = askdirectory(title = 'Select an experiment folder') # pops up a window to select your file
# uncomment this line if you use this jupyter notebook locally
#'G:/DREAM/from_github/PAMFluo/Experiments/2021-06-24_12_12_Ek_video'


In [24]:
show = True #option to output intermediary images in the segmentation process

# Import video file in HQ and select ROI
file_path = file_folder + "/video_1.tiff"
direc = os.path.split(file_path)[0]

# Initialize plotting tools
g = alienlab.plot.ShowFigure()
g.figsize = (15,7)
g.save_folder = "images"
g.date = False
p = alienlab.plot.PlotFigure()
p.figsize = (15,7)
p.save_folder = "images"
p.date = False

In [25]:
# read the stacked frame. dim = NxHxW (N images in the video, Heigt, Width)

frames_full = skimage.io.imread(file_path)

#frames_full = np.stack([frames_full[:,:,1]]*10, 0) 
#uncomment this line if you have a single RGB image. The [:,:,1] stands for selection of the green channel

FO = FramesOperator(frames_full)
im = normalize(FO.frames[0], 0, 1)
im = grey_to_rgb(im)*255

# CROP
#y, x = alienlab.io.select_roi(np.uint8(im)) #select area of interest
#FO.x = x
#FO.y = y
#FO.crop() #crop image

start_time = time.time()
FO.compute_stats() #compute various statistical values on the frames and the pixels
FO.normalize(0, 1)
print("--- Computed frames statistics in %04f seconds ---" % (time.time() - start_time))

#FO.global_stats: each array has size N, number of frames and represents the stats of each frame
#FO.frames_stats: each array has size FO.x, FO.y and is an image representing the N frames stats overlayed

if show:
    p.title = 'statistics'
    p.xlabel = 'frame number'
    p.ylabel = 'amplitude'
    p.label_list = ['max', 'min', 'mean', 'std']
    fig = p.plotting(np.asarray(FO.inds), [FO.global_stats['max'], 
                        FO.global_stats['min'], 
                        FO.global_stats['mean']])
    p.save_name = 'frames_stats'
    p.saving(fig)

''' IMAGE SEGMENTATION '''

# selection of the frames with high dynamics that will be used for the image segmentation process.
# Let M be the highest value taken by a pixel in all the frames of the video. The frame F is kept for processing only if at
# least one pixel in the frame F has a value above 0.8*M. 
FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.8)


--- Computed frames statistics in 0.334496 seconds ---


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:

plt.figure(figsize = (5, 5))
FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.98) # Select only images with high intensity to increase contrast and lower computation time


imref = FO.frames[FO.selected_inds].sum(axis = 0)
plt.imshow(imref, cmap = 'gray')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
def segment_image(contrast, autolevel, dist_max, dist_seg, disk_size, max_contrast, interact = True, showit = show):
    
    start_time = time.time()
    FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.98) # Select only images with high intensity to increase contrast and lower computation time

    #apply contrast filter to all frames
    frames_contrast = FO.apply(skimage.filters.rank.enhance_contrast,  selem = skimage.morphology.disk(contrast))
    #apply autolevel filter to all frames
    frames_autolevel = FO.apply(skimage.filters.rank.autolevel, selem = skimage.morphology.disk(autolevel))
    #sum the contrast images to get a reference grey-level contrast image
    frame_contrast = np.sum(frames_contrast, axis = 0)
    #sum the autolevel images to get a reference grey-level autolevel image
    frame_autolevel = np.sum(frames_autolevel, axis = 0)
    #obtain contrast mask from reference contrast image
    mask_contrast = make_binary(frame_contrast, soft_hard = 1)
    #otbain autolevel mask from reference autolevel image
    mask_autolevel =  make_binary(frame_autolevel, soft_hard = 1)
    #intersection of contrast aud autolevel masks
    mask_intersect = mask_contrast * mask_autolevel
    #clean the masks with a binary opening
    mask_intersect = skimage.morphology.binary_opening(mask_intersect, selem = skimage.morphology.disk(disk_size))
    #reference image of altitude for the watershed
    auto_contrast = normalize(mask_intersect * frame_autolevel)
    print("--- Computed binary mask in %04f seconds ---" % (time.time() - start_time))

    g.cmap = "inferno"
    if showit:
        g.figsize = (40,15)
        g.title_list =  'contrast', 'contrast threshold', 'mask intersect','autolevel', 'autolevel threshold','segmentation image'
        g.col_num = 3
        fig = g.multi([frame_contrast, mask_contrast, mask_intersect, 
                       frame_autolevel, mask_autolevel,  auto_contrast])
        g.save_name = 'Segmentation reference'
        g.saving(fig)

    start_time = time.time()
    ref = auto_contrast
    mask = mask_intersect
    #locate the local maxima
    local_maxi = alienlab.segment.local_maxima(auto_contrast, max_contrast, g,
                                                     ref_distance = dist_max, mask = mask, show = showit)
    #perform watershed segmentation
    watershed_im_mask = alienlab.segment.watershed(ref, mask, local_maxi,
                                                         g, ref_distance = dist_seg, show = False)
    segmented = watershed_im_mask
    print("--- Computed segmentation in %04f seconds ---" % (time.time() - start_time))

    if showit:
        alienlab.segment.show_segmentation(FO, segmented, g)
        
    if interact == False:
        return watershed_im_mask, FO


In [30]:
mask, FO = segment_image(contrast = 2, autolevel = 3, dist_max = True, dist_seg=True, disk_size = 2, max_contrast = 2, interact = False, showit= False)

--- Computed binary mask in 1.527745 seconds ---
--- Computed segmentation in 0.780458 seconds ---


In [32]:
g.cmap = "tab20"
g.figsize = (8, 5)
fig = g.multi(mask)
L, H  = np.shape(mask)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
# Collect item labels

# Item time trajectories with overlaps
# create a dictionnary with one entry for each item:
'''
{ '1.0': {'x_coords': np array, x coordinates in HQ}
            'y_coords': np array,  y coordinates in HQ
            'binned_coords': set, couples of (x,y) coordinates in binned video
            'surface': number of pixels in the item in HQ
            'pixel_values': array, size: (N, s) where N is number of frames and s surface
            'mean': array, size N, mean value of the item intensity for each frame
            'std':  array, size N, std value of the item intensity for each frame
            'remains' : True, the item is present in this segmentation step
            }
'2.0': {'x_coords'...
                }
    }
'''
segmented = mask
items = np.unique(segmented) #returns the set of values in items, corresponds to the values of the markers of local_maxima

items_dict = {}
for k in tqdm(items):
    key = str(k)
    items_dict[key] = {}
    x_coords, y_coords = np.nonzero(segmented == k)
    items_dict[key]['x_coords'] = x_coords
    items_dict[key]['y_coords'] = y_coords
    pixel_values = FO.frames[:,x_coords, y_coords]
    items_dict[key]['pixel_values'] = pixel_values
    items_dict[key]['surface'] = pixel_values.shape[1]
    items_dict[key]['mean'] = np.mean(pixel_values, axis = 1)
    items_dict[key]['std'] = np.std(pixel_values, axis = 1)
    items_dict[key]['remains'] = True




100%|█████████████████████████████████████████████████████████████████████████████| 1766/1766 [00:03<00:00, 581.31it/s]


In [35]:
data_sequence = {}
video_sequence = {}
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
for i in range(3):

    file_path = glob.glob(file_folder + "/*_ojip_curve_%d.csv"%i)[0] # pops up a window to select your file
    data_sequence[i] = pd.read_csv(file_path)
    data_sequence[i].plot( x = "time (s) o", y = "voltage (V) o", ax= axs[0][i%2])

    file_path = file_folder + "/video_%d.tiff"%i


    video = tiff.imread(file_path)
    video_sequence[i]=video
    axs[1][i%2].plot(np.mean(video, axis = (1,2)))
        
plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [116]:

NPQ_tot = []
video_algae = []
algae_list = list(items_dict.keys())
result_npq = copy.copy(mask)*0.0
for P in [5]:#[len(algae_list)]:#[8, 10, 12, 15, 18, 20, 25, 30, 35, 40, 60, 90, 120, 180, 200, 250, 300, 400, 500, 600, 700]:

    #for algae in random.sample(algae_list, P):
    for algae in tqdm(algae_list):
        #if int(algae):# in ['1', '80', '400', '250']:
            i = 0


            x_coords =  items_dict[algae]['x_coords']
            y_coords =  items_dict[algae]['y_coords']
            
            exposed = np.float(np.sum(video_sequence[2][0:5, x_coords, y_coords]))
            dark =  np.float(np.sum(video_sequence[0][0:5, x_coords, y_coords]))
            trace = np.mean(video_sequence[1][:, x_coords, y_coords], axis = (1))
            npq = (dark-exposed)/exposed
            video_algae.append(np.array(trace))
            NPQ_tot.append(npq)
          
            result_npq[mask==int(float(algae))] = npq

result_npq[result_npq != result_npq] = 1
plt.figure()
plt.imshow(result_npq)
plt.figure()
plt.imshow(FO.frames[FO.selected_inds].sum(axis = 0), cmap = 'gray')


  0%|                                                                                         | 0/1766 [00:00<?, ?it/s]<ipython-input-116-05f1a77d40b3>:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  exposed = np.float(np.sum(video_sequence[2][0:5, x_coords, y_coords]))
<ipython-input-116-05f1a77d40b3>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:
plt.imshow(result_npq)

In [118]:
plt.figure()
plt.hist(result_npq[mask != 0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.4944e+04, 2.5133e+04, 3.1900e+02, 6.2000e+01, 1.4600e+02,
        2.3000e+01, 1.2000e+01, 0.0000e+00, 2.8000e+01, 7.8000e+01]),
 array([-0.82178218, -0.28960396,  0.24257426,  0.77475248,  1.30693069,
         1.83910891,  2.37128713,  2.90346535,  3.43564356,  3.96782178,
         4.5       ]),
 <BarContainer object of 10 artists>)

In [119]:
x0 = [1.1, 8, 0.1]
def get_fit(decay, time):    

    parameters_estimated = optimize.least_squares(residuals,  x0, bounds = (0,1e8),
                                args = (time, decay, exp_decay))
    
    
    return np.array(parameters_estimated.x)


def make_fit(decay, time_array):
    plt.figure()
    params = get_fit(decay, time_array)
    plt.plot(time_array, exp_decay(params, time_array), label = params[1])
    plt.plot(time_array, decay, '.')
    plt.legend()
    
start = 1
decay = np.mean(video_sequence[1], axis = (1,2))[start:]
time_array = np.linspace(0, len(decay), len(decay))


#time_array = np.array(data_sequence[1]["time (s) o"].values)[30:-10]
#decay = np.array(data_sequence[1]["voltage (V) o"].values)[30:-10]

make_fit(decay, time_array)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [120]:
time_array = np.linspace(0, len(video_algae[i]), len(video_algae[i]))
params = Parallel(n_jobs = -1 )(delayed(get_fit)(video_algae[i][start:], time_array[start:]) for i in range(len(video_algae)))


In [121]:
params = np.array(params)
tau = params[:,1]
im_tau = np.copy(imref)*0
for algae in algae_list:
    algae = int(float(algae))
    im_tau[mask==algae] = tau[algae]

plt.figure()
plt.imshow(im_tau)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
plt.figure()
plt.hist(tau)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.607e+03, 1.000e+02, 3.700e+01, 1.000e+01, 1.000e+00, 6.000e+00,
        0.000e+00, 4.000e+00, 0.000e+00, 1.000e+00]),
 array([9.60696608e-02, 2.57669143e+04, 5.15337326e+04, 7.73005509e+04,
        1.03067369e+05, 1.28834187e+05, 1.54601006e+05, 1.80367824e+05,
        2.06134642e+05, 2.31901461e+05, 2.57668279e+05]),
 <BarContainer object of 10 artists>)

In [126]:

# Create a random image
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].imshow(im_tau)
axs[0].axis('off')

# Create and display textarea widget
txt = wdg.Textarea(
    value='',
    placeholder='',
    description='event:',
    disabled=False
)
display(txt)
coords = []

# Define a callback function that will update the textarea
def onclick(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata
    txt.value = str(event)#"x= %d, y = %d"%(ix, iy)

    global coords
    coords.append((ix, iy))
    
    algae_ind = mask[iy.astype(int), ix.astype(int)]
    if algae_ind != 0:

        decay = video_algae[algae_ind][1:]
        time_array = np.linspace(0, len(decay), len(decay))

        make_fit(decay, time_array)
        params = get_fit(decay, time_array)
        axs[1].plot(time_array, exp_decay(params, time_array), label = params[1])
        axs[1].plot(time_array, decay, '.')
        axs[1].legend()
        plt.tight_layout()
# Create an hard reference to the callback not to be cleared by the garbage collector
ka = fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Textarea(value='', description='event:', placeholder='')

In [114]:
algae_ind = mask[286, 226]
print(algae_ind)
decay = video_algae[algae_ind][1:]


0


In [110]:
plt.plot(decay)

In [112]:
#WAITTTTTTTTTTTTTTT

In [20]:
np.save("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_semihealthy_ter_09_15.npy", im_tau)
np.save("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_semihealthy_ter_09_15.npy", FO.frames[FO.selected_inds].sum(axis = 0))



# Compare

In [134]:
data_sequence.keys()

dict_keys([0])

In [229]:
import matplotlib.cm as cm
folder_list = glob.glob("G:/DREAM/from_github/PAMFluo/Experiments/*qE_OJIP")
color = cm.tab10(np.tile(np.linspace(0, 1, 10), 50))
data_sequence = {}
video_sequence = {}
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
for j, file_folder in enumerate(folder_list[5:]):
    if j in [12, 14]:# not in [3, 4, 5]:
        print(file_folder)
        for i in range(0,3):

            file_path = glob.glob(file_folder + "/*_ojip_curve_%d.csv"%i)[0] # pops up a window to select your file
            data_sequence[i] = pd.read_csv(file_path)
            x = data_sequence[i]["time (s) o"]
            y = data_sequence[i]["voltage (V) o"]
            axs[0][i%2].plot(x[:-3], y[:-3]/y.max(), label = os.path.split(file_folder)[1], color = color[j])
            file_path = file_folder + "/video_%d.tiff"%i


            video = tiff.imread(file_path)
            video_sequence[i]=video
            v = np.mean(video[:-1], axis = (1,2))
            axs[1][i%2].plot(v/v.max(), color = color[j])
axs[0][1].legend()    
plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

G:/DREAM/from_github/PAMFluo/Experiments\2021-09-16_15_37_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-16_16_00_qE_OJIP


[]

In [190]:
j

19

In [147]:
video/video.max()

array([[[0.   , 0.   , 0.125, ..., 0.   , 0.125, 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        ...,
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.125, 0.   , 0.   ],
        [0.   , 0.125, 0.   , ..., 0.   , 0.   , 0.   ]],

       [[0.125, 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        ...,
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ]],

       [[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.125],
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        ...,
        [0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , ..., 0.   , 0. 

In [94]:

fig, axs = plt.subplots(2, 3, figsize=(14, 7))

vmin = 1
vmax = 20

healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_healthy_09_15.npy")
axs[0][0].imshow(clip(healthy), vmin=vmin, vmax=vmax)
healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_healthy_09_15.npy")
axs[1][0].imshow(clip(healthy_base), cmap = "gray")

sick = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_sick_09_15.npy")
axs[0][1].imshow(clip(sick), vmin=vmin, vmax=vmax)
sick_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_sick_09_15.npy")
axs[1][1].imshow(clip(sick_base), cmap ="gray")


sick = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_semihealthy_09_15.npy")
imcb = axs[0][2].imshow(clip(sick), vmin=vmin, vmax=vmax)
sick_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_semihealthy_09_15.npy")
axs[1][2].imshow(clip(sick_base), cmap ="gray")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(imcb, cax=cbar_ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:

fig, axs = plt.subplots(2, 3, figsize=(14, 7))

vmin = 1
vmax = 20

healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_semihealthy_09_15.npy")
axs[0][0].imshow(clip(healthy), vmin=vmin, vmax=vmax)
healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_semihealthy_09_15.npy")
axs[1][0].imshow(clip(healthy_base), cmap = "gray")

sick = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_semihealthy_bis_09_15.npy")
axs[0][1].imshow(clip(sick), vmin=vmin, vmax=vmax)
sick_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_semihealthy_bis_09_15.npy")
axs[1][1].imshow(clip(sick_base), cmap ="gray")


sick = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_semihealthy_ter_09_15.npy")
imcb = axs[0][2].imshow(clip(sick), vmin=vmin, vmax=vmax)
sick_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_semihealthy_ter_09_15.npy")
axs[1][2].imshow(clip(sick_base), cmap ="gray")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(imcb, cax=cbar_ax)

<ipython-input-21-cf672a71c238>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(2, 3, figsize=(14, 7))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …